Дано: лог nginx,в архиве gz.

Надо: разобраться с форматом лога и преобразовать в формат csv. Т.е. чтобы параметры в строках были разделены точкой с запятой.

Зачем: это дает возможность анализировать лог в программах, которые могут читать csv (excel, библиотека pandas)

Подсказки: в python можно использовать
библиотеку gzip для чтения архива,
библиотеку re для обработки строк с помощью регулярных выражений

In [14]:
import gzip, re
import gzip

inputFile = 'csv_example.csv.gz'
outputFile = 'csv_example.csv'

In [37]:
with gzip.open(inputFile, 'rb') as f:
    with open(outputFile, 'w')as out:
        cnt = 0;
        for line in f:
            if (cnt == 10):
                break
            line = line.decode().strip()
            #lineSplit = re.split(r'(\w{3}\s+\d+)\s+(\d{2}:\d{2}:\d{2})\s+(.+?)\s+(.+?)\s+(\d{2}\/\w{3}\/\d{4}:\d{2}:\d{2}:\d{2})\s+(.+?)\s+(.+?)\s+-\s+(.+?)\s+(.+?)\s+(.+?)\s+"(.+?)"\s+(.+?)\s+(.+?)\s+(.+?)\s+"(.+?)"\s+"(.+?)"\s+-\s+(.+?)\s+(.+?)\s+', line)
            line = re.sub(r';','\;',line)
            line = re.split(r'"',line)
            for i in range(0, len(line)):
                if (i % 2 == 0):
                    word = re.sub(r'\s+-*\s*',';',line[i])
                    out.write(word)
                    print(word)
                else:
                    out.write('"' + line[i] + '"')
                    print(line[i])
            out.write('\n')
            cnt += 1

Aug;4;00:00:06;p00esianlbext01;esia.gosuslugi.ru_https_access:;03/Aug/2014:23:59:57;+0400;82.196.148.114;GET;/idp/resources/img/theme/sprite_arrow.png;HTTP/1.1;
200
;0.001;1407095997.008;1434;
https://esia.gosuslugi.ru/idp/resources/css/theme.css
;
Firefox/3.6.13, TestNVG
;upstream_responce:;0.001
Aug;4;00:00:06;p00esianlbext01;esia.gosuslugi.ru_https_access:;03/Aug/2014:23:59:57;+0400;109.165.86.241;GET;/idp/AuthnEngine;HTTP/1.1;
302
;0.011;1407095997.009;311;
http://www.gosuslugi.ru/pgu/personcab
;
Mozilla/5.0 (X11\; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36
;upstream_responce:;0.011
Aug;4;00:00:06;p00esianlbext01;esia.gosuslugi.ru_https_access:;03/Aug/2014:23:59:57;+0400;82.196.148.114;GET;/idp/resources/img/theme/primefaces/bg_paginator.png;HTTP/1.1;
200
;0.002;1407095997.009;3498;
https://esia.gosuslugi.ru/idp/resources/css/theme.css
;
Firefox/3.6.13, TestNVG
;upstream_responce:;0.002
Aug;4;00:00:06;p00esianlbext01;esia.gosuslugi.ru_ht